In [1]:
import truss
from pathlib import Path
import requests

c:\Users\vincencius\anaconda3\envs\aixiety\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
#Containerize Model
tr = truss.load(Path("/llama-2-7b-aixiety"))
# command = tr.docker_build_setup(build_dir=Path("./llama-2-7b-aixiety-v2"))

print(tr)

In [10]:
tr.docker_build_setup(build_dir=Path("./llama-2-7b-aixiety"))

'docker build llama-2-7b-aixiety -t custom-model:latest'

In [19]:
BOS, EOS = "<s>", "</s>"
B_INST, E_INST = "[INST]", "[/INST]"
B_SYS, E_SYS = "<<SYS>>\n", "\n<</SYS>>\n\n"
DEFAULT_SYSTEM_PROMPT = """\
Anda adalah psikolog berlisensi, harap berikan pasien ini tanggapan yang bermanfaat untuk kekhawatiran mereka. 
Pastikan mereka merasa nyaman dengan bertanya untuk memahami masalah mereka terlebih dahulu. 
Jika topik berada diluar ranah psikolog, alihkan pembicaraan mengenai kesehatan mental pasien."""


def format_to_llama_chat_style(history) -> str:
    # history is now a list of dictionaries with "instruction" and "response" keys
    prompt = ""
    current_length = 0

    for i, dialog in enumerate(history[:-1]):
        print("Urutan ke-", i)
        instruction, response = dialog["instruction"], dialog["response"]

        # Calculate the length of the current dialog
        dialog_length = len(instruction.strip()) + len(response.strip())

        # Check if adding this dialog would exceed the maximum length
        if current_length + dialog_length > 1024:
            break  # Stop adding more dialogues if the limit is reached

        # prepend system instruction before first instruction
        if i == 0:
            instruction = f"{B_SYS}{DEFAULT_SYSTEM_PROMPT}{E_SYS}" + instruction
        else:
            # the tokenizer automatically adds a bos_token during encoding,
            # for this reason the bos_token is not added for the first instruction
            prompt += BOS
        prompt += f"{B_INST} {instruction.strip()} {E_INST} {response.strip()} " + EOS

    # new instruction from the user
    new_instruction = history[-1]["instruction"].strip()

    # the tokenizer automatically adds a bos_token during encoding,
    # for this reason the bos_token is not added for the first instruction
    if len(history) > 1:
        prompt += BOS
    else:
        # prepend system instruction before the first instruction
        new_instruction = f"{B_SYS}{DEFAULT_SYSTEM_PROMPT}{E_SYS}" + new_instruction

    prompt += f"{B_INST} {new_instruction} {E_INST}"
    return prompt

In [8]:
history = {'history': [
    {
        'instruction': "gua merasa sedih belakangan ini karena diputusin",
        'response': 'Penting untuk diingat bahwa putusan adalah bagian dari kehidupan, dan penting untuk mencari dukungan saat Anda mengalami emosi yang sulit. Mari kita bekerja mengembangkan strategi untuk membantu Anda mengatasi perasaan Anda dan menemukan cara yang sehat untuk mengekspresikan emosi Anda.'
    },
    {
        'instruction': 'berapa 1+1?',
        'response': ''
    }
]}

# print(history)
# history = history.pop("history")
# prompt = format_to_llama_chat_style(history)
# print(prompt)

In [9]:
res = requests.post("http://34.66.79.181:8080/v1/models/model:predict", json=history)
res.close()
print(res.json())

{'data': '  Sangat baik! 1 + 1 adalah 2. 😊'}
